In [8]:
import numpy as np
import pandas as pd
import os, re, operator
from pprint import pprint

from src.utils import *
from src.k_nearest_neighbors import *
from src.logistic_regression import *
from src.naive_bayes import *
from src.neutral_network import *
#from src.support_vector_machines import *
from src.validation import *

PATH_POSITIVE_TRUTHFUL  = 'op_spam_v1.4/positive/truthful/'
PATH_POSITIVE_DECEPTIVE = 'op_spam_v1.4/positive/deceptive/'
PATH_NEGATIVE_TRUTHFUL  = 'op_spam_v1.4/negative/truthful/'
PATH_NEGATIVE_DECEPTIVE = 'op_spam_v1.4/negative/deceptive/'
ALL_PATH = [PATH_POSITIVE_TRUTHFUL, PATH_POSITIVE_DECEPTIVE,
           PATH_NEGATIVE_TRUTHFUL, PATH_NEGATIVE_DECEPTIVE]

In [9]:
# Remoção de Stop Words
import nltk
from nltk.corpus import stopwords

# Download do dicionário de stop wordsgreat
nltk.download('stopwords')

def stopwords_removal(tokens_list):
    new_tokens_list = []
    
    stop_words = stopwords.words('english')
    new_tokens_list = [token for token in tokens_list if token not in stop_words]
    
    return new_tokens_list

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\matna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# N-gram dos tokens
def generate_ngram(tokens_list, gram1, gram2):
    # N-grams list
    allNGrams = []
    
    #N-Gram
    for idx in range(gram1, gram2):
        ngrams = zip(*[tokens_list[i:] for i in range(idx)])
        allNGrams += ([" ".join(ngram) for ngram in ngrams])
    
#     print(allNGrams)
    return allNGrams

In [4]:
# Remoção de features irrelevantes por threshold
def features_removal(features_dic, threshold):
    new_features_dic = {}

    #Remove values < threshold
    for key, value in features_dic.items():
        if value > threshold:
            new_features_dic[key] = value

    new_features_list = [k for k, v in new_features_dic.items()]
    new_features_dic = sorted(new_features_dic.items(), key=operator.itemgetter(1))
    
    return new_features_list, new_features_dic

In [ ]:
# # Pre processamento
# def pre_processing(paths, stopwords=False, ngram=False, threshold=0):
#     tokens, tokens_dic = tokenize(paths)
    
#     if (stopwords):
#         tokens, tokens_dic = stopwords_removal(tokens)
    
#     if (ngram):
#         tokens, tokens_dic = generate_ngram(tokens, 1, 3)
    
#     tokens, tokens_dic = features_removal(tokens_dic, threshold)
    
#     return tokens, tokens_dic

# # posTruthWordSplited, posTruthDic, listWords = wordsProcessed([PATH_POSITIVE_DECEPTIVE, PATH_NEGATIVE_DECEPTIVE], 7)
# # posTruthWordSplited, posTruthDic, listWords = wordsProcessed([PATH_POSITIVE_DECEPTIVE], 3)
# posTruthDic, listWords = pre_processing([PATH_NEGATIVE_DECEPTIVE])

# #pprint(posTruthDic)
# print(len(posTruthDic))

In [6]:
# Tokenização dos textos
def pre_processing(paths, stopwords=False, ngram=False, threshold=0):
    token_dic = {}
    dirs =[]
    
    for p in paths:
        text_path = [p + t for t in os.listdir(p)]
        dirs += (text_path)
    
    for texts in dirs:
        with open(texts, 'r', encoding='utf-8') as stream:
            tokens = []
            text = stream.read()

            #Pre processing
            tokens = [word for word in (re.sub(r'[^\w\s]+','', text.replace('\n','')).lower().split(' ')) if word != '']
            
            if stopwords:
                tokens = stopwords_removal(tokens)
            
            if ngram:
                tokens = generate_ngram(tokens, 1, 3)    

            #Creating dictonary for counting words
            for token in tokens:
                if token in token_dic:
                    token_dic[token] += 1
                else:
                    token_dic[token] = 1
    
    token_list, token_dic = features_removal(token_dic, threshold)
#     pprint(token_dic)
    return token_list, token_dic

# posTruthWordSplited, posTruthDic, listWords = wordsProcessed([PATH_POSITIVE_DECEPTIVE, PATH_NEGATIVE_DECEPTIVE], 7)
# posTruthWordSplited, posTruthDic, listWords = wordsProcessed([PATH_POSITIVE_DECEPTIVE], 3)
posTruthDic, listWords = pre_processing([PATH_NEGATIVE_DECEPTIVE], stopwords=True, threshold=3)

#pprint(posTruthDic)
print(len(posTruthDic))

4966


In [ ]:
matrix = []
Y = []

def generateFeatures(path, example_class):
    i = 1
    for texts in os.listdir(path):
        i+=1
        print(i)
        aux = np.zeros(len(listWords))
        with open(path + texts, 'r', encoding='utf-8') as stream:
            #Pre-pre processing
            text = stream.read()
            allNgrams = []
            wordSplited = [word for word in (re.sub(r'[^\w\s]+','', text.replace('\n','')).lower().split(' ')) if word != '']

            stop_words = stopwords.words('english')
            wordSplited = [word for word in wordSplited if word not in stop_words]
            
            #N-Gram
            for index in range(2, 4):
                ngrams = zip(*[wordSplited[i:] for i in range(index)])
                allNgrams += ([" ".join(ngram) for ngram in ngrams])
            
            for idx, word in enumerate(listWords):
                for ngram in allNgrams:
                    if word == ngram:
                        aux[idx] += 1

            Y.append(example_class)
            matrix.append(aux)
            
generateFeatures(PATH_NEGATIVE_DECEPTIVE, 1)
generateFeatures(PATH_NEGATIVE_TRUTHFUL, 0)
generateFeatures(PATH_POSITIVE_DECEPTIVE, 1)
generateFeatures(PATH_POSITIVE_TRUTHFUL, 0)

matrix = np.array(matrix)
Y = np.array(Y)
matrix_norm, mu, sigma = normalize(matrix)

#save(np.column_stack((matrix_norm, matrix[:, -1])), listWords)
print('ACABOOOOOOO')

In [ ]:
if matrix_norm.shape[1] != len(posTruthDic):
    raise Exception('Tamanhos diferentes!')

In [ ]:
# a = matrix_norm[400:720]
# b = matrix_norm[0:320]
# X2 = np.append(a,b, axis=0)

# a = Y[400:720]
# b = Y[0:320]
# Y2 = np.append(a,b, axis=0)

# a = matrix_norm[720:800]
# b = matrix_norm[320:400]
# X_val = np.append(a,b, axis=0)

# a = Y[720:800]
# b = Y[320:400]
# Y_val = np.append(a,b, axis=0)

#custo, gamma = svm(X2, Y2, X_val, Y_val)
#print(custo, '\t', gamma)

In [ ]:
# y = knn(X_val[81], X2, Y2, 5)
# #print(Y_val[81], ' ',y,' ', ind_viz)
# print(y)

In [ ]:
folds = stratified_kfolds(Y, 5, np.unique(Y))

for train_index, test_index in folds:
    print('Train: ',len(train_index))
    print('Test: ',len(test_index))

In [ ]:
# import random

# resultados = []
# for x in range(0, 5):
#     Y_resultado = np.array([random.randint(0, 1) for i in range(0, 160)])
#     Y_classes = np.array([random.randint(0, 1) for i in range(0, 160)])
#     classes = [0, 1]

#     cm = get_confusionMatrix(Y_classes, Y_resultado, classes)
    
    
#     resultado = relatorioDesempenho(cm, classes, False)
#     resultados.append(resultado)

# mediaFolds(resultados, classes)

In [ ]:
# NAIVE BAYES

resultados = []
classes = np.unique(Y)

for train_index, test_index in folds:
    # Treinamento
    probsPos, probsNeg = calcularProbabilidades(matrix[train_index], Y[train_index])
    
    # Classificação
    pred = []
    for x in matrix[test_index]:
        probPos, probNeg = classificacao(x, probsPos, probsNeg, sum(Y == 1)/len(Y), sum(Y == 0)/len(Y))
        if (probPos >= probNeg):
            pred.append(1)
        else:
            pred.append(0)

    cm = get_confusionMatrix(Y[test_index], pred, np.unique(Y))
    resultado = relatorioDesempenho(cm, classes, False)
    
    resultados.append(resultado)

mediaFolds(resultados, classes)